# Decorators and the sometimes useful (useless) but often interesting

## Decorators example: Debugging with the print statement
Let's start with a simple function that does something:
```python
def sum(x,y):
    print x+y;
    return x+y;
```
Ok, but you probably want to remove the print before you let someone see your code... and maybe changes in the future will also need this debugging step. Our first 'decorator'
```python
def debug(val):
    print val;
    return val;
def sum(x,y):
    return x+y;
x,y=2,3;
# 'explicit decorator'
debug(sum(x,y))
```
Ok but we moved the print statement -out- of the function. Version controling between the master and debugging branch might still be weird. What we really want is a modified version of the function sum. What if a function could modify the call to sum:
```python
def debug_on(func):
    def debugging(*args,**kwargs):
        print 'Scope: debugging',func(*args,**kwargs);
        return func(*args,**kwargs);
    print 'Scope: debug_on',debugging
    return debugging
    
```
Read what is happening here carefully. 
```python
def debug_on(func):
    def debugging(*args,**kwargs):
    ...
    return debugging;
```
We take in a function, and return a function. What happens for example:
```python
>> a = debug_on(sum); # a->debugging
                      # Note that debugging calls our original function sum
Scope: debug_on <function debugging at 0xf4b4d3ac>
```
We pass all positional (\*args) and keyword (\*\*kwargs) through debugging to sum and return the result. 
```python
>> print(a(3,4));
Scope: debugging, 7
7
```

In [ ]:
def debug_on(func):
    def debugging(*args,**kwargs):
        retval = func(*args,**kwargs);
        print('Scope: debugging %s:%s:%s'%(func,func.__name__,retval));
        return retval;
    print('Scope: debug_on',debugging)
    return debugging

In [ ]:
# the @ syntax is equivalent to function nesting https://www.python.org/dev/peps/pep-0318/
# debug_on(sum)

@debug_on
def sum(x,y):        # sum->debugging
    return x+y;
@debug_on
def sum2(x,y):       #sum2->debugging, but seperate instance
    return (x+y)**2.;
print('Scope: main sum %s, sum2 %s'%(sum,sum2))


### NOTE! decorated sum and debugging point to the same function!

In [ ]:
sum(3,4)
sum2(3,4)

## What about class or a class functions?
An example class:
```python
class point():
    def __init__(self,x,y):
        self.x=x;
        self.y=y;
    def dist(self,x,y):
        return self.x**2.+self.y**2;
```

In [ ]:
@debug_on
class point():
    @debug_on
    def __init__(self,x,y):
        self.x=x;
        self.y=y;
    @debug_on
    def dist(self):
        return self.x**2.+self.y**2;

In [ ]:
a = point(1,2);
a.dist()

### So it works on classes too. Why?
(Left for the reader)
### Ok, so the syntax for debugging the class is kind of annoying... can we debug all the methods with a single @?
```python
def debug_on_classes(cls):
    def debugging(*args,**kwargs):
        cdict = cls.__dict__;
        for item in cdict:
            func = getattr(cls,item);
            if( hasattr(func,'__call__') ):
                setattr(cls,item,debug_on(func))
        return cls(*args,**kwargs);
    return debugging;
```
Note the careful syntax. Here we simply ask for all the items in the class dictionary and, if they are callable, we wrap then with the debug_on decorator. Test it out!

In [ ]:
def debug_on_classes(cls):
    def debugging(*args,**kwargs):
        cdict = cls.__dict__;
        print(cdict);
        for item in cdict:
            func = getattr(cls,item);
            if( hasattr(func,'__call__') ):
                setattr(cls,item,debug_on(func))
        return cls(*args,**kwargs);
    return debugging;


In [ ]:
@debug_on_classes
class point():
    def __init__(self,x,y):
        self.x=x;
        self.y=y;
    def dist(self):
        return self.x**2.+self.y**2;
a = point(1,2);
a.dist();

### Decorator: Main ideas
1. Functions can be made to point other places... and maybe eventually calling the original code as written.
2. Pointers!POINTERS!
3. @ really just nests the function calls, the power to call the original function is yours!


## Metaclasses: A higher calling
So decorators are ways to modify class and function behavior by redirecting calls to a wrapper function first. How can there be a higher level of control? Metaclasses. These control the behavior of classes before instantiation.

More to come...
